# bw2analyzer exploration functions

This notebook shows some new exploration functions addedd to `bw2analyzer`.

In [1]:
import bw2data as bd
import bw2analyzer as ba
import bw2calc as bc
import bw2io as bi

We use `ecoinvent` as an example database, but the functions shown here are generic.

In [2]:
bd.projects.set_current("ecoinvent 3.7.1 bw2")

## `print_recursive_supply_chain`

Sometimes it is convenient to print the supply chain of an activity. This function is only for exploration; use `bw2calc.GraphTraversal` in production.

In [3]:
ei = bd.Database("ecoinvent 3.7.1")

In [4]:
act = bd.get_activity(('ecoinvent 3.7.1', 'b28714960dbd0334840bbcecaf2c88c8'))
act

'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, PT, None)

In [5]:
ba.print_recursive_supply_chain(act)

1: 'heat and power co-generation, natural gas, conventional power plant, 
  0.246: 'market group for natural gas, high pressure' (cubic meter, Europe wit
    0.00324: 'market for natural gas, high pressure' (cubic meter, AT, None)
    0.0141: 'market for natural gas, high pressure' (cubic meter, BE, None)
    0.00525: 'market for natural gas, high pressure' (cubic meter, CZ, None)
    0.0496: 'market for natural gas, high pressure' (cubic meter, DE, None)
    0.000666: 'market for natural gas, high pressure' (cubic meter, DK, None)
    0.00749: 'market for natural gas, high pressure' (cubic meter, ES, None)
    0.00166: 'market for natural gas, high pressure' (cubic meter, FI, None)
    0.0145: 'market for natural gas, high pressure' (cubic meter, FR, None)
    0.036: 'market for natural gas, high pressure' (cubic meter, GB, None)
    0.0013: 'market for natural gas, high pressure' (cubic meter, GR, None)
    0.0028: 'market for natural gas, high pressure' (cubic meter, HU, None)
    0

This function also supports:

* Using a custom string instead of tabs for indentation
* Writing to a file-like object instead of printing to `stdout`
* Rescaling all amounts to an arbitrary value
* Using a cutoff to limit what is returned
* Specifying an arbitrary maximum recursion depth

Here is a silly example of most of these options:

In [6]:
ba.print_recursive_supply_chain(act, max_level=5, cutoff=0.02, tab_character="🐶", amount=2)

2: 'heat and power co-generation, natural gas, conventional power plant, 
🐶0.492: 'market group for natural gas, high pressure' (cubic meter, Europe wit
🐶🐶0.0282: 'market for natural gas, high pressure' (cubic meter, BE, None)
🐶🐶0.0993: 'market for natural gas, high pressure' (cubic meter, DE, None)
🐶🐶🐶0.0604: 'transport, pipeline, long distance, natural gas' (ton kilometer, DE, 
🐶🐶🐶0.0213: 'natural gas, high pressure, import from NL' (cubic meter, DE, None)
🐶🐶🐶0.0379: 'natural gas, high pressure, import from RU' (cubic meter, DE, None)
🐶🐶🐶🐶0.127: 'transport, pipeline, long distance, natural gas' (ton kilometer, RU, 
🐶🐶🐶🐶0.0332: 'transport, pipeline, long distance, natural gas' (ton kilometer, RER 
🐶🐶🐶🐶0.0379: 'natural gas production' (cubic meter, RU, None)
🐶🐶🐶🐶🐶0.0376: 'market for drying, natural gas' (cubic meter, GLO, None)
🐶🐶🐶0.0319: 'natural gas, high pressure, import from NO' (cubic meter, DE, None)
🐶🐶🐶🐶0.0319: 'petroleum and gas production, off-shore' (cubic meter, NO, None)
🐶🐶

## `print_recursive_calculation`

We can do the same thing, but filter not by the amounts consumed but their respective environmental impacts.

In [7]:
ipcc = ('IPCC 2013', 'climate change', 'GWP 100a')

In [8]:
ba.print_recursive_calculation(act, ipcc)

Fraction of score | Absolute score | Amount | Activity
0001 | 0.609 |     1 | 'heat and power co-generation, natural gas, conventional power plant, 
  0.148 | 0.09043 | 0.2459 | 'market group for natural gas, high pressure' (cubic meter, Europe wit
    0.0102 | 0.006202 | 0.01408 | 'market for natural gas, high pressure' (cubic meter, BE, None)
    0.0393 | 0.02394 | 0.04963 | 'market for natural gas, high pressure' (cubic meter, DE, None)
      0.0244 | 0.01485 | 0.01894 | 'natural gas, high pressure, import from RU' (cubic meter, DE, None)
    0.0241 | 0.0147 | 0.02472 | 'market for natural gas, high pressure' (cubic meter, IT, None)
      0.0176 | 0.01075 | 0.01325 | 'natural gas, high pressure, import from RU' (cubic meter, IT, None)
    0.0127 | 0.007755 | 0.05454 | 'market for natural gas, high pressure' (cubic meter, NO, None)


This function supports the same arguments as `print_recursive_supply_chain`.

## `find_differences_in_inputs`

Some databases have multiple activities that seem similar, but it is hard to tell how different they really are. `find_differences_in_inputs` is one of three functions that helps distinguish between different activities. It will look through the database that the activity came from, find all other activities with the same name and reference product, and see if their inputs are substantially different.

In our example activity, there are 60 different activities with the same name and reference product, so we limit the result to certain locations.

In [9]:
canada = ['CA-BC', 'CA-AB', 'CA-NS', 'CA-MB', 'CA-QC', 'CA-PE', 'CA-ON', 'CA-NB', 'CA-SK', 'CA-NT']

In [10]:
result = ba.find_differences_in_inputs(act, locations=canada)
result

{'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, PT, None): {'natural gas, high pressure': 0.245885675976663,
  'water, completely softened': 0.0575372481785392,
  'water, decarbonised': 1.91790827261797,
  'residue from cooling tower': -9.58954136308986e-06,
  'Hexane': 7.60450630093026e-06,
  'Sulfur dioxide': 5.4852176596874e-06,
  'Water': 0.05863365240771915,
  'Acetic acid': 1.16033450493388e-06,
  'Formaldehyde': 3.09742186027802e-07,
  'Butane': 8.87991530222121e-06,
  'Propane': 6.76062666097835e-06,
  'Carbon monoxide, fossil': 0.000191790827261797,
  'Carbon dioxide, fossil': 0.512081508788999,
  'Dinitrogen monoxide': 9.36898191173883e-06,
  'Water, cooling, unspecified natural origin': 0.0566582068869226,
  'Particulates, < 2.5 um': 4.70846480927712e-06,
  'PAH, polycyclic aromatic hydrocarbons': 7.67163309047189e-08,
  'Nitrogen oxides': 0.000351936168025398,
  'Propionic acid': 1.53432661809438e-07,
  'Pentane': 1.1

This function compares the net amount of each flow, and adds up multiple exchanges which reference the same flow. The above printed result is still a bit hard to interpret; returning this analysis result as a dataframe can allow for quick interpretation of the results:

In [29]:
ba.find_differences_in_inputs(act, locations=canada, as_dataframe=True)

,"natural gas, high pressure","water, completely softened","water, decarbonised",residue from cooling tower,Hexane,Sulfur dioxide,Water,Acetic acid,Formaldehyde,Butane,...,"Carbon dioxide, fossil",Dinitrogen monoxide,"Water, cooling, unspecified natural origin","Particulates, < 2.5 um","PAH, polycyclic aromatic hydrocarbons",Nitrogen oxides,Propionic acid,Pentane,Ethane,"Methane, fossil"
location,,,,,,,,,,,,,,,,,,,,,
PT,0.245886,0.057537,1.917908,-0.000010,0.000008,0.000005,0.058634,0.000001,3.097422e-07,0.000009,...,0.512082,0.000009,0.056658,0.000005,7.671633e-08,0.000352,1.534327e-07,0.000011,0.000013,0.000009
CA-NB,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-SK,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-MB,0.246926,0.057781,1.926020,-0.000010,0.000008,0.000006,0.058882,0.000001,3.110523e-07,0.000009,...,0.514247,0.000009,0.056898,0.000005,NaN,0.000353,NaN,0.000011,0.000013,0.000009
CA-NT,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-AB,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-PE,0.246926,0.057781,1.926020,-0.000010,0.000008,0.000006,0.058882,0.000001,3.110523e-07,0.000009,...,0.514247,0.000009,0.056898,0.000005,NaN,0.000353,NaN,0.000011,0.000013,0.000009
CA-ON,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009
CA-QC,0.240821,0.056352,1.878406,-0.000009,0.000007,0.000005,0.057426,0.000001,3.033626e-07,0.000009,...,0.501534,0.000009,0.055491,0.000005,7.513625e-08,0.000345,1.502725e-07,0.000011,0.000013,0.000009


It is even easier to see clear patterns when the data is normalized:

In [14]:
df / df.iloc[0]

,"natural gas, high pressure","water, completely softened","water, decarbonised",residue from cooling tower,Hexane,Sulfur dioxide,Water,Acetic acid,Formaldehyde,Butane,...,"Carbon dioxide, fossil",Dinitrogen monoxide,"Water, cooling, unspecified natural origin","Particulates, < 2.5 um","PAH, polycyclic aromatic hydrocarbons",Nitrogen oxides,Propionic acid,Pentane,Ethane,"Methane, fossil"
location,,,,,,,,,,,,,,,,,,,,,
PT,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
CA-NB,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-MB,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,...,1.004230,1.004230,1.004230,1.004230,NaN,1.004230,NaN,1.004230,1.004230,1.004230
CA-NS,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,...,1.004230,1.004230,1.004230,1.004230,NaN,1.004230,NaN,1.004230,1.004230,1.004230
CA-AB,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-BC,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-ON,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-QC,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,...,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404,0.979404
CA-PE,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,1.004230,...,1.004230,1.004230,1.004230,1.004230,NaN,1.004230,NaN,1.004230,1.004230,1.004230


## `compare_activities_by_lcia_score`

A common question when trying to choose between different activities with similar sounding names is: Does it matter which one we choose? Are they actually any different? `compare_activities_by_lcia_score` allows for a comparison of any activities. If we look at very similar activities, we don't see a real difference:

In [31]:
justin = [
    a for a in ei 
    if a['name'] == act['name'] 
    and a['reference product'] == act['reference product']
    and a['location'] in canada
]
justin

['heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-SK, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NT, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-PE, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-AB, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-QC, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NB, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-MB, None),
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-ON, None),
 'heat and power co-generation, natural gas, con

In [33]:
ba.compare_activities_by_lcia_score(
    [
        a for a in justin 
        if a['location'] != 'CA-QC'  # Je me souviens ;)
    ],
    ipcc
)

All activities similar


Allowing for Québec already produces different results:

In [34]:
ba.compare_activities_by_lcia_score(
    justin,
    ipcc
)

Differences observed. LCA scores:
	0.564 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-SK, None)
	0.564 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NT, None)
	0.578 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-PE, None)
	0.556 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-AB, None)
	0.706 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-QC, None)
	0.564 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-NB, None)
	0.578 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, CA-MB, None)
	0.564 -> 'heat and power co-generation, natural gas, conventional power plan

Note that the Brightway developers take no position on Québec, Canada, Justin, or the correctness of the results provided above!

You can set the cutoff for what a "real" difference is with the `band` argument. Just for fun, let's look at many different CHP activities.

In [17]:
{act['name'] for act in ei if act['name'].startswith('heat and power co-generation, natural gas')}

{'heat and power co-generation, natural gas, 160kW electrical, Jakobsberg',
 'heat and power co-generation, natural gas, 160kW electrical, lambda=1',
 'heat and power co-generation, natural gas, 1MW electrical, lean burn',
 'heat and power co-generation, natural gas, 200kW electrical, lean burn',
 'heat and power co-generation, natural gas, 500kW electrical, lean burn',
 'heat and power co-generation, natural gas, 50kW electrical, lean burn',
 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical',
 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical',
 'heat and power co-generation, natural gas, mini-plant 2KW electrical'}

In [35]:
ba.compare_activities_by_lcia_score(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ], 
    ipcc,
    band=1
)

All activities similar


In [18]:
ba.compare_activities_by_lcia_score(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ], 
    ipcc,
    band=0.25
)

Differences observed. LCA scores:
	0.615 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, GR, None)
	0.360 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, IR, None)
	0.627 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, HU, None)
	0.608 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, SK, None)
	0.516 -> 'heat and power co-generation, natural gas, combined cycle power plant, 400MW electrical' (kilowatt hour, LV, None)
	0.562 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, SI, None)
	0.699 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW electrical' (kilowatt hour, TR, None)
	0.688 -> 'heat and power co-generation, natural gas, conventional power plant, 100MW elec

## `compare_activities_by_grouped_leaves`

Knowing that there are differences isn't always super helpful - one wants to *why* there are differences. We can't really go through the supply chains of many different activities manually, and keep all the differences straight. The function `compare_activities_by_grouped_leaves` will sum the *impacts* of the different supply chain activities by their [Common Product Classification](https://unstats.un.org/unsd/classifications/Econ/cpc) code. This will only work on databases which provide CPC codes in the same format as ecoinvent.

This function has multiple output formats. The default is a Python tuple of `(labels, data)`, but this is not shown here, as it is intended for programmatic use. Humans would choose either an HTML table:

In [21]:
ba.compare_activities_by_grouped_leaves(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ][:5], 
    ipcc,
    output_format="html",
)

Omitting activity name common prefix: 'heat and power co-generation, natural gas, '


activity,product,location,unit,total,direct emissions,"12020: Natural gas, liquefied or in the gaseous st",65131: Transport services via pipeline of petroleu,17100: Electrical energy,17300: Steam and hot water,53262: Power plants,65212: Coastal and transoceanic water transport se,39990: Other wastes n.e.c.,86211: Support services to oil and gas extraction,8621: Support services to mining,"4124: Bars and rods, hot-rolled, of iron or steel",53241: Long-distance pipelines,15310: Natural sands,54330: Excavating and earthmoving services,"34710: Polymers of ethylene, in primary forms","4128: Tubes, pipes and hollow profiles, of steel",53251: Local pipelines,531: Buildings,"374: Plaster, lime and cement",18000: Natural water,6424: Air transport services of passengers,"39270: Waste, parings and scrap of plastics"
"conventional power plant, 100MW electrical",,US-NPCC,kilowatt hour,0.590,0.818,0.097,0.013,0.036,0.021,0.007,0.000,0.002,0.002,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"conventional power plant, 100MW electrical",,EE,kilowatt hour,0.563,0.847,0.093,0.047,0.000,0.001,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.001,0.000,0.000,0.000,0.000,0.000
"combined cycle power plant, 400MW electrical",,RoW,kilowatt hour,0.554,0.837,0.093,0.020,0.011,0.000,0.004,0.006,0.000,0.004,0.000,0.003,0.001,0.001,0.001,0.001,0.001,0.000,0.001,0.000,0.000,0.000,0.000
"combined cycle power plant, 400MW electrical",,KR,kilowatt hour,0.492,0.880,0.083,0.000,0.012,0.000,0.003,0.002,0.006,0.006,0.005,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"combined cycle power plant, 400MW electrical",,US-WECC,kilowatt hour,0.435,0.820,0.097,0.013,0.037,0.021,0.005,0.000,0.002,0.002,0.000,0.000,0.003,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


Or as a pandas `DataFrame`:

In [22]:
df = ba.compare_activities_by_grouped_leaves(
    [a for a in ei 
     if a['name'].startswith('heat and power co-generation, natural gas')
     and a['reference product'] == act['reference product']
    ][:5], 
    ipcc,
    output_format="pandas",
)
df

Omitting activity name common prefix: 'heat and power co-generation, natural gas, '


,activity,product,location,unit,total,direct emissions,17100: Electrical energy,65131: Transport services via pipeline of petroleum and natural gas,"12020: Natural gas, liquefied or in the gaseous state",17300: Steam and hot water,8621: Support services to mining,86211: Support services to oil and gas extraction,53262: Power plants,39990: Other wastes n.e.c.,53241: Long-distance pipelines,65212: Coastal and transoceanic water transport services of freight by tankers,53251: Local pipelines,18000: Natural water,39910: Municipal waste
0,"conventional power plant, 100MW electrical",,IT,kilowatt hour,0.686884,0.775616,0.081627,0.048086,0.045734,0.001161,0.010525,0.011150,0.004583,0.002157,0.003408,0.000407,0.000705,0.000242,0.000112
1,"combined cycle power plant, 400MW electrical",,HU,kilowatt hour,0.627076,0.689001,0.113822,0.096907,0.050409,0.001160,0.017464,0.013518,0.001168,0.002902,0.004554,0.000000,0.001518,0.000215,0.000186
2,"conventional power plant, 100MW electrical",,US-FRCC,kilowatt hour,0.590461,0.817801,0.036404,0.012908,0.096847,0.020536,0.000000,0.002113,0.007319,0.001556,0.002603,0.000000,0.000388,0.000259,0.000000
3,"conventional power plant, 100MW electrical",,CA-NB,kilowatt hour,0.563542,0.896115,0.019113,0.045732,0.024807,0.000000,0.000000,0.002101,0.007472,0.001457,0.001792,0.000000,0.000363,0.000146,0.000000
4,"conventional power plant, 100MW electrical",,TW,kilowatt hour,0.488965,0.877332,0.012010,0.000000,0.082746,0.000000,0.005302,0.005740,0.005966,0.006390,0.000000,0.002335,0.000332,0.000274,0.000000


What is included in this result? Direct emissions are those coming from the functional unit. Then, the function goes through the supply chain of each activity, and applies a cutoff criteria. If a given input matches the cutoff criteria (either it is too deep in the supply chain, or its impact is too small), we look up that inputs CPC code, and add it to any existing inputs with the same CPC code. We can also add impacts from direct emissions of an activity, even if it doesn't meet the cutoff criteria (if we didn't do this, the shares wouldn't sum to one).

As in the above functions, you can control how deep the search goes with the `max_level` and `cutoff` arguments. You can also switch from the default result type, which is fractional share of total impact, to absolute impact amounts, with the `mode` argument.

You can also use this function to explore the supply chain of a single activity. This is helpful when databases like ecoinvent provide many similar input activities (e.g. many electricity providers); it can be helpful to group by the product classification.

In [26]:
offset_printing = bd.get_activity(('ecoinvent 3.7.1', 'ff315bbf13be4ced226551a12cfdca53'))
offset_printing

'offset printing, per kg printed paper' (kilogram, CH, None)

In [27]:
ba.compare_activities_by_grouped_leaves(
    [offset_printing], 
    ipcc,
    output_format="html",
)

Omitting activity name common prefix: 'offset printing, per kg printed '


activity,product,location,unit,total,direct emissions,32113: Mechanical wood pulp; semi-chemical wood pu,41431: Unwrought aluminium,17100: Electrical energy,34790: Other plastics in primary forms; ion exchan,17300: Steam and hot water,6511: Road transport services of freight,32151: Corrugated paper and paperboard,34: Basic chemicals,"32112: Chemical wood pulp, other than dissolving g",15400: Clays,23220: Starches; inulin; wheat gluten; dextrins an,34240: Phosphates of triammonium; salts and peroxy,341: Basic organic chemicals,031: Wood in the rough,34231: Chemical elements n.e.c.; inorganic acids e,11040: Brown coal briquettes and similar solid fue,"12020: Natural gas, liquefied or in the gaseous st",35110: Paints and varnishes and related products,342: Basic inorganic chemicals n.e.c.,4153: Semi-finished products of aluminium or alumi,53269: Other constructions for manufacturing,6512: Railway transport services of freight,33370: Fuel oils n.e.c.,"34210: Hydrogen, nitrogen, oxygen, carbon dioxide",69120: Gas distribution through mains (on own acco,65229: Other inland water transport services of fr,161: Chemical and fertilizer minerals,"37420: Quicklime, slaked lime and hydraulic lime","34139: Other alcohols, phenols, phenol-alcohols, a",34280: Hydrogen peroxide; phosphides; carbides; hy,"34710: Polymers of ethylene, in primary forms",14290: Other non-ferrous metal ores and concentrat,11010: Hard coal,266: Woven fabrics (except special fabrics) of cot,"21611: Soya bean oil, crude",39283: Non-agglomerated wood waste and scrap,39950: Wastes from chemical or allied industries,33350: White spirit and special boiling point indu,39920: Sewage sludge,532: Civil engineering works,"21651: Palm oil, crude","34170: Ethers, alcohol peroxides, ether peroxides,",65119: Other road transport services of freight,34540: Oils and other products of the distillation,34800: Synthetic rubber and factice derived from o,"36330: Plates, sheets, film, foil and strip, of pl",53252: Local cables and related works,"444: Machinery for mining, quarrying and construct",39: Wastes or scraps,35130: Printing ink,"89200: Moulding, pressing, stamping, extruding and",39910: Municipal waste
paper,,CH,kilogram,2.092,0.000,0.227,0.162,0.135,0.058,0.043,0.040,0.031,0.029,0.028,0.024,0.021,0.019,0.018,0.017,0.016,0.015,0.014,0.011,0.009,0.008,0.007,0.006,0.005,0.005,0.004,0.003,0.003,0.003,0.002,0.002,0.002,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.011


In this particular instance, it was interesting for me to see the high fraction of impact coming from use of aluinium, which (probably) isn't the first thing one thinks of when considering printing.